In [1]:
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
from autogen import AssistantAgent
import os

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [2]:
# MongoDB connection details
MONGO_URI = os.environ["MONGODB_URI"]  # Replace with your MongoDB URI
DB_NAME = "vector_mflix1"
COLLECTION_NAME = "movies_with_vectors"

In [3]:
# Initialize MongoDB client
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

In [4]:
# Load pre-trained embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [5]:
def embed_plot(plot_text):
    """Generate vector embedding for the input plot."""
    return embedding_model.encode(plot_text).tolist()

In [10]:
def find_closest_movie(plot_embedding, top_k=5):
    """Find the movie with the closest embedding."""
    pipeline = [
        {
            "$addFields": {
                "embedding_distance": {
                    "$let": {
                        "vars": {"plot_emb": "$plot_embedding"},
                        "in": {
                            "$sqrt": {
                                "$sum": [
                                    {"$pow": [{"$subtract": [e, {"$arrayElemAt": ["$$plot_emb", i]}]}, 2]}
                                    for i, e in enumerate(plot_embedding)
                                ]
                            }
                        },
                    }
                }
            }
        },
        {"$sort": {"embedding_distance": 1}},
        {"$limit": top_k}
    ]
    return list(collection.aggregate(pipeline))

In [24]:
# Create the AutoGen Assistant Agent
class MovieFinderAssistant(AssistantAgent):
    def find_movie(self, user_input):
        """Process user input and find the closest movie."""
        try:
            # Step 1: Embed the input plot
            plot_embedding = embed_plot(user_input)
            
            # Step 2: Query the database for the closest movie
            result = find_closest_movie(plot_embedding, top_k=5)
            response = ''
            
            # Step 3: Format and return the response
            if result:
                for i, movie in enumerate(result):
                    response += f"{i}. Closest movie: {movie['title']} ({movie['year']})\nGenres: {', '.join(movie['genres'])}\nPlot: {movie['plot']}"
            else:
                response = "No matching movie found."
        except Exception as e:
            response = f"An error occurred: {e}"
        
        return response

In [25]:
# Instantiate the assistant agent
assistant = MovieFinderAssistant(name="Movie Finder", description="Finds movies based on plot descriptions.")

In [26]:
# User interaction loop
while True:
    user_input = input("Enter a plot description (or 'exit' to quit): ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
    print(assistant.find_movie(user_input))

0. Closest movie: L'opèra de quat'sous (1931)
Genres: Comedy, Crime, Musical
Plot: 1. Closest movie: The Song of Night (1932)
Genres: Comedy, Musical
Plot: 2. Closest movie: The World Moves On (1934)
Genres: Drama, History, Romance
Plot: 3. Closest movie: David Golder (1931)
Genres: Drama
Plot: 4. Closest movie: The Red Head (1932)
Genres: Drama
Plot: 
Goodbye!
